In [29]:
from math import sqrt
import pandas as pd
import numpy as np
from gurobipy import *
#import matplotlib.pyplot as plt

In [30]:
data = pd.read_csv('ER.csv', names=['PD','ERL','loan_amnt','int_rate','ER'])
data.head()

,PD,ERL,loan_amnt,int_rate,ER
0,0.004935,0.001974,15000.0,0.0532,0.051226
1,0.109972,0.043989,17000.0,0.0749,0.030911
2,0.038825,0.015530,20000.0,0.0532,0.037670
3,0.040959,0.016384,16000.0,0.1274,0.111016
4,0.159881,0.063952,2000.0,0.1699,0.105948


In [31]:
N_loans=len(data.index)
loan_ID = list(range(0,N_loans))

In [32]:
# Create an empty model
m = Model('portfolio')

In [33]:
# Add a variable for each loan
vars = pd.Series(m.addVars(loan_ID), index=loan_ID)

In [61]:
# Objective is to maximize expected return ER.
portfolio_ER = vars.dot(data['ER'])
m.setObjective(portfolio_ER, GRB.MAXIMIZE)

In [62]:
# Fix budget with a constraint
m.addConstr(vars.sum() == 5000000, 'budget')
m.addConstrs((vars[i] <= 500000 for i in range(N_loans)), name='diversification')
m.addConstrs((vars[i] <= data['loan_amnt'][i] for i in range(N_loans)), name='loan_size')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [63]:
m.printStats()


Statistics for model portfolio_copy :
  Linear constraint matrix    : 193560 Constrs, 96779 Vars, 387116 NZs
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 2.88e-05, 0.308853 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 1000, 5e+06 ]


In [43]:
# Optimize model to find the minimum risk portfolio
m.optimize()

Optimize a model with 193560 rows, 96779 columns and 387116 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-05, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 5e+06]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 193559 rows and 10567 columns
Presolve time: 2.23s
Presolved: 1 rows, 86212 columns, 86212 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00 (roughly 34 MBytes of memory)
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.77712802e+09  5.87625281e+08  3.62e+09 4.44e-16  1.23e+05     3s
   1   7.67238817e+07  5.23899399e+08  8.92e+07 3.11e-15  5.80e+03     3s
   2   1.23893995e+07  1.33056338e+08  1.19e+07 2.66e-15  1.03e+03     3s

Barrier performed 2 iteratio

In [59]:
#data['investment']=vars
#data.head()
#len(vars)
vars.sum()
#for v in vars: data['investment']=v.lb

AttributeError: 'Series' object has no attribute 'lb'

In [55]:
data.head()

,PD,ERL,loan_amnt,int_rate,ER,investment
0,0.004935,0.001974,15000.0,0.0532,0.051226,0.0
1,0.109972,0.043989,17000.0,0.0749,0.030911,0.0
2,0.038825,0.015530,20000.0,0.0532,0.037670,0.0
3,0.040959,0.016384,16000.0,0.1274,0.111016,0.0
4,0.159881,0.063952,2000.0,0.1699,0.105948,0.0


In [51]:
np.savetxt("Logistic_portfolio.csv", data, delimiter=",")

In [57]:
#avg_ret=data['loan_amnt'].dot(data['int_rate'])/sum(data['loan_amnt'])
sum(data['investment'])
#log_ret=data['investment'].dot(data['int_rate'])/sum(data['investment'])

0.0

In [ ]:
# Create an expression representing the expected return for the portfolio
portfolio_return = stock_return.dot(vars)

# Display minimum risk portfolio
print('Minimum Risk Portfolio:\n')
for v in vars:
    if v.x > 0:
        print('\t%s\t: %g' % (v.varname, v.x))
minrisk_volatility = sqrt(portfolio_risk.getValue())
print('\nVolatility      = %g' % minrisk_volatility)
minrisk_return = portfolio_return.getValue()
print('Expected Return = %g' % minrisk_return)

In [ ]:


# Add (redundant) target return constraint
target = m.addConstr(portfolio_return == minrisk_return, 'target')

# Solve for efficient frontier by varying target return
frontier = pd.Series()
for r in np.linspace(stock_return.min(), stock_return.max(), 100):
    target.rhs = r
    m.optimize()
    frontier.loc[sqrt(portfolio_risk.getValue())] = r

# Plot volatility versus expected return for individual stocks
ax = plt.gca()
ax.scatter(x=stock_volatility, y=stock_return,
           color='Blue', label='Individual Stocks')
for i, stock in enumerate(stocks):
    ax.annotate(stock, (stock_volatility[i], stock_return[i]))

# Plot volatility versus expected return for minimum risk portfolio
ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
            horizontalalignment='right')

# Plot efficient frontier
frontier.plot(color='DarkGreen', label='Efficient Frontier', ax=ax)

# Format and display the final plot
ax.axis([0.005, 0.06, -0.02, 0.025])
ax.set_xlabel('Volatility (standard deviation)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid()
plt.show()